In [5]:
# 🌌 Daily Catalyst - Aesthetic Animated To-Do App
# Built using Tkinter + SQLite + ttkbootstrap
# Features: Galaxy theme, task tracking, motivation quotes, progress tracker, and more

import tkinter as tk
from tkinter import ttk, messagebox
from tkcalendar import DateEntry
import sqlite3
import random
import datetime
import math

# -------------------------------------------------------
# DATABASE SETUP
# -------------------------------------------------------
def init_db():
    conn = sqlite3.connect("daily_catalyst.db")
    cur = conn.cursor()
    cur.execute("""
    CREATE TABLE IF NOT EXISTS tasks (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        task TEXT NOT NULL,
        due_date TEXT,
        motivation TEXT,
        completed INTEGER DEFAULT 0
    )
    """)
    conn.commit()
    conn.close()


# -------------------------------------------------------
# MAIN APPLICATION
# -------------------------------------------------------
class DailyCatalystApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🌌 Daily Catalyst")
        self.root.geometry("980x660")
        self.root.resizable(False, False)

        # ---- COLORS AND BACKGROUND ----
        self.bg_colors = ["#1f1d36", "#3f3351", "#864879", "#e9a6a6"]
        self.color_index = 0

        self.bg_canvas = tk.Canvas(root, width=980, height=660, highlightthickness=0)
        self.bg_canvas.pack(fill="both", expand=True)

        # 🌠 Stars for animation
        self.stars = []
        for _ in range(50):
            x, y = random.randint(0, 980), random.randint(0, 660)
            s = self.bg_canvas.create_oval(x, y, x + 2, y + 2, fill="white", outline="")
            self.stars.append(s)
        self.animate_stars()

        # ---- SIDEBAR ----
        self.sidebar = tk.Frame(root, bg="#292841", width=180, height=660)
        self.sidebar.place(x=0, y=0)
        tk.Label(self.sidebar, text="☀ Control Panel", bg="#292841", fg="#F2E9E4",
                 font=("Century Gothic", 13, "bold")).pack(pady=15)

        ttk.Button(self.sidebar, text="🌈 Switch Theme", command=self.switch_theme).pack(pady=8)
        ttk.Button(self.sidebar, text="📊 Task Insights", command=self.show_progress).pack(pady=8)
        ttk.Button(self.sidebar, text="💡 Inspire Me", command=self.show_new_quote).pack(pady=8)
        ttk.Button(self.sidebar, text="❌ Exit", command=root.quit).pack(pady=8)

        # ---- HEADER ----
        self.title = tk.Label(root, text="✨ Daily Catalyst", font=("Century Gothic", 26, "bold"),
                              bg=self.bg_colors[0], fg="white")
        self.title.place(x=320, y=25)

        self.quotes = [
            "Small steps every day lead to big change 🌱",
            "Discipline is the bridge between goals and success 💪",
            "Be your own inspiration 🌠",
            "Progress, not perfection 🌸",
            "Stay focused, stay consistent, stay kind 🪷",
            "You can do hard things 🔥"
        ]
        self.quote_label = tk.Label(root, text=random.choice(self.quotes),
                                    font=("Century Gothic", 12), fg="#FFD6EC", bg=self.bg_colors[0])
        self.quote_label.place(x=220, y=80)

        # ---- TASK ENTRY ----
        self.task_frame = tk.Frame(root, bg=self.bg_colors[0])
        self.task_frame.place(x=200, y=120, width=760, height=90)

        tk.Label(self.task_frame, text="Task:", bg=self.bg_colors[0], fg="white",
                 font=("Century Gothic", 12)).grid(row=0, column=0, padx=10, pady=10)
        self.task_entry = tk.Entry(self.task_frame, width=35, font=("Century Gothic", 12))
        self.task_entry.grid(row=0, column=1, padx=10)

        tk.Label(self.task_frame, text="Due Date:", bg=self.bg_colors[0], fg="white",
                 font=("Century Gothic", 12)).grid(row=1, column=0)
        self.due_entry = DateEntry(self.task_frame, width=15, background='darkblue',
                                   foreground='white', borderwidth=2, date_pattern='yyyy-mm-dd')
        self.due_entry.grid(row=1, column=1, sticky="w", padx=10)

        tk.Label(self.task_frame, text="Motivation:", bg=self.bg_colors[0], fg="white",
                 font=("Century Gothic", 12)).grid(row=0, column=2, padx=10)
        self.motivation_entry = tk.Entry(self.task_frame, width=20, font=("Century Gothic", 12))
        self.motivation_entry.grid(row=0, column=3, padx=10)

        ttk.Button(self.task_frame, text="➕ Add", command=self.add_task).grid(row=1, column=2, padx=10)
        ttk.Button(self.task_frame, text="🗑 Delete", command=self.delete_task).grid(row=1, column=3, padx=10)

        # ---- TASK LIST ----
        columns = ("ID", "Task", "Due Date", "Motivation", "Completed")
        self.tree = ttk.Treeview(root, columns=columns, show="headings", height=15)
        for col in columns:
            self.tree.heading(col, text=col)
            if col == "Task":
                self.tree.column(col, width=260)
            elif col == "Completed":
                self.tree.column(col, width=100, anchor="center")
            else:
                self.tree.column(col, width=140)
        self.tree.place(x=200, y=240, width=760, height=370)
        self.tree.tag_configure("completed", background="#b8f5b1", foreground="black")
        self.tree.bind("<Double-1>", self.toggle_completed)

        self.load_tasks()

        # Start animated background after widgets exist
        self.animate_background()

    # -------------------------------------------------------
    # 🌟 ANIMATIONS
    # -------------------------------------------------------
    def animate_stars(self):
        for s in self.stars:
            x, y, x1, y1 = self.bg_canvas.coords(s)
            dy = random.choice([1, 2])
            if y1 > 660:
                y = 0
            self.bg_canvas.move(s, 0, dy)
        self.root.after(100, self.animate_stars)

    def animate_background(self):
        next_color = self.bg_colors[self.color_index]
        self.root.configure(bg=next_color)
        self.bg_canvas.configure(bg=next_color)
        self.sidebar.configure(bg=next_color)
        self.title.configure(bg=next_color)
        self.quote_label.configure(bg=next_color)
        self.task_frame.configure(bg=next_color)
        self.color_index = (self.color_index + 1) % len(self.bg_colors)
        self.root.after(3500, self.animate_background)

    # -------------------------------------------------------
    # 📝 TASK MANAGEMENT
    # -------------------------------------------------------
    def add_task(self):
        task = self.task_entry.get().strip()
        date = self.due_entry.get()
        motivation = self.motivation_entry.get().strip()

        if not task:
            messagebox.showwarning("Missing Info", "Please enter a task name.")
            return

        conn = sqlite3.connect("daily_catalyst.db")
        cur = conn.cursor()
        cur.execute("INSERT INTO tasks (task, due_date, motivation, completed) VALUES (?, ?, ?, 0)",
                    (task, date, motivation))
        conn.commit()
        conn.close()

        self.task_entry.delete(0, tk.END)
        self.motivation_entry.delete(0, tk.END)
        self.load_tasks()

    def load_tasks(self):
        for row in self.tree.get_children():
            self.tree.delete(row)
        conn = sqlite3.connect("daily_catalyst.db")
        cur = conn.cursor()
        cur.execute("SELECT * FROM tasks")
        rows = cur.fetchall()
        conn.close()
        for row in rows:
            tag = "completed" if row[4] == 1 else ""
            completed_symbol = "✅" if row[4] == 1 else "❌"
            self.tree.insert("", tk.END, values=(row[0], row[1], row[2], row[3], completed_symbol), tags=(tag,))

    def delete_task(self):
        selected = self.tree.selection()
        if not selected:
            messagebox.showwarning("No Selection", "Select a task to delete.")
            return
        task_id = self.tree.item(selected, "values")[0]
        conn = sqlite3.connect("daily_catalyst.db")
        cur = conn.cursor()
        cur.execute("DELETE FROM tasks WHERE id=?", (task_id,))
        conn.commit()
        conn.close()
        self.load_tasks()

    def toggle_completed(self, event):
        selected = self.tree.selection()
        if not selected:
            return
        task_id = self.tree.item(selected, "values")[0]
        completed_symbol = self.tree.item(selected, "values")[4]
        new_status = 0 if completed_symbol == "✅" else 1

        conn = sqlite3.connect("daily_catalyst.db")
        cur = conn.cursor()
        cur.execute("UPDATE tasks SET completed=? WHERE id=?", (new_status, task_id))
        conn.commit()
        conn.close()
        self.load_tasks()

    # -------------------------------------------------------
    # 🎨 SIDEBAR FUNCTIONS
    # -------------------------------------------------------
    def switch_theme(self):
        random.shuffle(self.bg_colors)
        self.load_tasks()

    def show_progress(self):
        conn = sqlite3.connect("daily_catalyst.db")
        cur = conn.cursor()
        cur.execute("SELECT COUNT(*), SUM(completed) FROM tasks")
        total, done = cur.fetchone()
        conn.close()
        if not total:
            progress = 0
        else:
            progress = int((done or 0) / total * 100)
        messagebox.showinfo("📊 Progress", f"Tasks Completed: {progress}%")

    def show_new_quote(self):
        self.quote_label.config(text=random.choice(self.quotes))


# -------------------------------------------------------
# RUN APP
# -------------------------------------------------------
if __name__ == "__main__":
    init_db()
    root = tk.Tk()
    app = DailyCatalystApp(root)
    root.mainloop()
